# Riduzione del dataset

Lo scopo del seguente notebook è ridurre la quantità di audio per etichetta, cercando di mantenendo il più possibile le caratteristiche di sbilanciamento del dataset originale.

# Installazione e importazione delle librerie necessarie

Come di consueto, si procede con l'installazione e la sucessiva importazione delle librerie necessarie per questa fase. In questo caso, Tensorflow servirà per l'estrazione del dataset, mentre le restanti saranno utilizate la manipolazione del dataset

In [1]:
import tensorflow as tf
import os
import random
import shutil

## Estrazione dataset
La funzione `audio_dataset_from_directory(directory)` rappresenta un modo semplice e veloce per ottenere informazioni dalle **sottodirectory di un dataset**. Il suo funzionamento di base è simile ad altre funzioni viste durante il corso, come per esempio `image_dataset_from_directory`. Nel caso specifico, il dataset è costituito da **30 sottodirectory**, ognuna corrispondente a una delle **30 classi** rappresentate come **indici numerici**, con una **batch_size di 32**.

In [2]:
# estraiamo il nostro dataset originale
dataset = tf.keras.utils.audio_dataset_from_directory(directory='../original/train/audio')

Found 64721 files belonging to 30 classes.


In [3]:
# stampiamo il numero delle classi e i nomi delle classi del dataset
print('Numero classi:', len(dataset.class_names))
print('Classi:',dataset.class_names)

Numero classi: 30
Classi: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


## Funzioni utilizzate
Andiamo a creare le funzioni che ci serviranno per la riduzione del dataset.

La prima funzione, denominata `copy_files`, ha il compito di creare la directory di destinazione per il dataset ridotto e copiare i file selezionati. La seconda funzione, chiamata `downsample_class`, riduce il dataset, mantenendo in questo caso una quantità di audio proporzionato alla quantità iniziale.

L'idea iniziale era di campionare 300 audio per ogni classe, ma si è giunti alla soluzione di estrarre un campione in modo casuale anziché un numero costante, così da poter preservare lo sbilanciamento del dataset originale.

In [4]:
# funzione per copiare i file
def copy_files(files, src_dir, dest_dir):
    if not os.path.exists(dest_dir): # se la directory di destinazione non esiste allora la creo
        os.makedirs(dest_dir)
    for file in files: # copio ogni file che mi viene passato nella directory di destinazione, quindi nella directory della classe
        shutil.copy(os.path.join(src_dir, file), os.path.join(dest_dir, file))

# funzione per diminuire la quantità di audio per label
def downsample_class(class_dir, class_name, target_dir, ratio):
    files = os.listdir(class_dir) # mi salvo tutti i file della classe in una variabile
    num_files_to_keep = int(len(files) * ratio) # calcolo la quantità di file che devo mantenere per la classe in questione
    files_to_keep = random.sample(files, num_files_to_keep) # scelgo randomicamente i file da mantenere
    
    # chiamo la funzione precedentemente dichiarata per copiare i file nella directory
    copy_files(files_to_keep, class_dir, os.path.join(target_dir, class_name))

Utilizzando le funzioni appena create, otteniamo un dataset che è il 20% delle dimensioni di quello originale, su cui verranno eseguiti i passaggi successivi. Il set si riduce da una **dimensione iniziale di 1.90 GB contenente 64721 audio totali** a una **dimensione finale di 389 MB con un totale 12933 audio totali**.

In [5]:
original_dir = '../original/train/audio' # path dataset originale
downsampled_dir = '../reduced_dataset/dataset/audio' # path dataset ridotto

classes = [dir for dir in os.listdir(original_dir) if os.path.isdir(os.path.join(original_dir, dir))]

for class_name in classes:
    class_dir = os.path.join(original_dir, class_name)
    downsample_class(class_dir, class_name, downsampled_dir, 0.20)